<a href="https://colab.research.google.com/github/albertsalgueda/tensorflow/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/joanby/tensorflow2/blob/master/Collab%206%20-%20Aprendizaje%20por%20Transferencia%20y%20puesta%20a%20punto%20de%20parámetros%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

## Paso 1: Instalación de dependencias y configuración del entorno en GPU

In [ ]:
#!pip install tensorflow-gpu==2.0.0.alpha0
%tensorflow_version 2.x

In [1]:
!pip install tqdm

### Descargar el dataset de Perros vs Gatos

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2022-02-19 20:55:26--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.129.128, 209.85.234.128, 108.177.112.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.129.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   209MB/s    in 0.3s    

2022-02-19 20:55:26 (209 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Paso 2: Pre procesado del data set

### Importar las dependencias del proyecto

In [3]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.8.0'

### Descomprimir el dataset de Perros vs Gatos

In [4]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [5]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [6]:
zip_object.extractall("./")

In [7]:
zip_object.close()

### Configurar las rutas al dataset

In [8]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [9]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Construir el Modelo

\### Cargar un modelo pre entrenado (MobileNetV2)

Tenemos que adaptar el tamaño de las imagenes para que estén igual que el modelo que entrenó google

In [10]:
IMG_SHAPE = (128, 128, 3)

consultar los parametros del modelo de Google...
include_top = False --> voy a cambiar la parte final de la red neural. 

In [11]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

9420800/9406464 [==============================] - 0s 0us/step


In [12]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

### Congelar el modelo base

congelar las capas base de la red neuronal básica y cambiar las del final

In [13]:
base_model.trainable = False

### Definir la cabecera personalizada para nuestra red neuronal

In [14]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

Reducir toda la información  del output haciendo un average

In [15]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [16]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [17]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

### Definir el modelo

In [18]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']           

### Compilar el modelo

lr --> learning rate

In [20]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


### Crear generadores de datos

Redimensionar imágenes

    Las grandes arquitecturas pre-entrenadas solamente soportan cierto tipo de tamaños de imágenes.
    
Por ejemplo: MobileNet (la arquitectura que nosotros usamos) soporta: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [22]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [23]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [24]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


### Entrenar el modelo 

In [33]:
model.fit_generator(train_generator, epochs=10, validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/10
16/16 [==============================] - 10s 623ms/step - loss: 0.2895 - accuracy: 0.8955 - val_loss: 0.2915 - val_accuracy: 0.9070
Epoch 2/10
16/16 [==============================] - 10s 607ms/step - loss: 0.2651 - accuracy: 0.9065 - val_loss: 0.2687 - val_accuracy: 0.9160
Epoch 3/10
16/16 [==============================] - 9s 597ms/step - loss: 0.2445 - accuracy: 0.9145 - val_loss: 0.2496 - val_accuracy: 0.9240
Epoch 4/10
16/16 [==============================] - 9s 594ms/step - loss: 0.2272 - accuracy: 0.9200 - val_loss: 0.2336 - val_accuracy: 0.9270
Epoch 5/10
16/16 [==============================] - 10s 601ms/step - loss: 0.2121 - accuracy: 0.9275 - val_loss: 0.2197 - val_accuracy: 0.9300
Epoch 6/10
16/16 [==============================] - 9s 594ms/step - loss: 0.1995 - accuracy: 0.9310 - val_loss: 0.2074 - val_accuracy: 0.9340
Epoch 7/10
16/16 [==============================] - 9s 593ms/step - loss: 0.1882 - accuracy: 0.9345 - val_loss: 0.1971 - val_accuracy: 0.9340
Epo

### Evaluar el modelo de aprendizaje por transferencia

In [32]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


In [27]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.8939999938011169


## Puesta a punto de parámetros


Un par de cosas:

- NUNCA HAY QUE USAR la puesta a punto (fine tuning) de parámetros en toda la red neuronal: con algunas de las capas superiores (las finales) es más que suficiente . En la mayoría de casos, son las más especializadas. El objetivo del fine tuning es adaptar esa parte específica de la red neuronal para nuestro nuevo dataset específico.
- Empezar con la puesta a punto DESPUÉS de haber finalizado la fase de aprendizaje por transferencia. Si intentamos hacer el Fine tuning inmediatamente, los gradientes serán muy diferentes entre nuestra cabecera personalizada de la red neuronal y las nuevas capas no congeladas del modelo base.

### Descongelar unas cuantas capas superiores del modelo

In [28]:
base_model.trainable = True

In [29]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 154


In [30]:
fine_tune_at = 100

vamos a delimitar el numero de redes neurales que vamos a poder entrenar,  en este caso las 100 primeras capas no las vamos a tocar

In [31]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compilar el modelo para la puesta a punto

In [34]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


### Puesta a punto

In [35]:
model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/5
16/16 [==============================] - 18s 707ms/step - loss: 0.1571 - accuracy: 0.9420 - val_loss: 0.0668 - val_accuracy: 0.9770
Epoch 2/5
16/16 [==============================] - 10s 608ms/step - loss: 0.0190 - accuracy: 0.9990 - val_loss: 0.0704 - val_accuracy: 0.9760
Epoch 3/5
16/16 [==============================] - 10s 609ms/step - loss: 0.0099 - accuracy: 0.9995 - val_loss: 0.0737 - val_accuracy: 0.9780
Epoch 4/5
16/16 [==============================] - 10s 609ms/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 0.0790 - val_accuracy: 0.9730
Epoch 5/5
16/16 [==============================] - 10s 611ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.1224 - val_accuracy: 0.9690


### Evaluar el modelo re calibrado

In [36]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


In [37]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.968999981880188
